<h2><center>STAGE 3 - CREATING FACTS</center></h2>

---

In [1]:
import os
import pandas as pd

In [2]:
from utils.data_warehouse_mappings import add_surrogate_key_dimension_table, add_surrogate_key_fact_table, \
    map_surrogate_to_natural_key, save_lookup_to_json

In [3]:
from utils.constants import fact_tables_folder, fact_order_line_file, fact_order_header_file

In [4]:
from utils.constants import processed_dataset_folder, customers_processed_dataset, geolocation_processed_dataset, \
    items_processed_dataset, payments_processed_dataset, reviews_processed_dataset, orders_processed_dataset, \
        products_processed_dataset, sellers_processed_dataset, category_processed_dataset

In [5]:
from utils.constants import lookup_tables_folder, lookup_table_customer, lookup_table_seller, lookup_table_product, \
    lookup_table_customer_geolocation, lookup_table_seller_geolocation, lookup_table_order_indicator, lookup_table_date, \
        lookup_table_order_number

In [6]:
from utils.constants import dimension_tables_folder, dim_customer_file, dim_seller_file, dim_product_file, dim_geolocation_file, \
    dim_customer_geolocation_file, dim_seller_geolocation_file, dim_order_indicator_file, dim_date_file

<h3>1. READING THE PROCESSED CSV FILES FROM STAGE 1</h3>

In [7]:
customers = pd.read_csv(os.path.join(processed_dataset_folder, customers_processed_dataset))
geolocation = pd.read_csv(os.path.join(processed_dataset_folder, geolocation_processed_dataset))
items = pd.read_csv(os.path.join(processed_dataset_folder, items_processed_dataset))
payments = pd.read_csv(os.path.join(processed_dataset_folder, payments_processed_dataset))
reviews = pd.read_csv(os.path.join(processed_dataset_folder, reviews_processed_dataset))
orders = pd.read_csv(os.path.join(processed_dataset_folder, orders_processed_dataset))
products = pd.read_csv(os.path.join(processed_dataset_folder, products_processed_dataset))
sellers = pd.read_csv(os.path.join(processed_dataset_folder, sellers_processed_dataset))
category = pd.read_csv(os.path.join(processed_dataset_folder, category_processed_dataset))

<h3>2. READING THE DIMENSION CSV FILES FROM STAGE 2</h3>

In [8]:
dim_customer = pd.read_csv(os.path.join(dimension_tables_folder, dim_customer_file))
dim_seller = pd.read_csv(os.path.join(dimension_tables_folder, dim_seller_file))
dim_product = pd.read_csv(os.path.join(dimension_tables_folder, dim_product_file))
dim_geolocation = pd.read_csv(os.path.join(dimension_tables_folder, dim_geolocation_file))
dim_customer_geolocation = pd.read_csv(os.path.join(dimension_tables_folder, dim_customer_geolocation_file))
dim_seller_geolocation = pd.read_csv(os.path.join(dimension_tables_folder, dim_seller_geolocation_file))
dim_order_indicator = pd.read_csv(os.path.join(dimension_tables_folder, dim_order_indicator_file))
dim_date = pd.read_csv(os.path.join(dimension_tables_folder, dim_date_file))

<h3>3. CREATING DATAFRAMES FOR EACH FACT TABLE</h3>

<h3>Fact Table <sup>factOrderLineAccumulating</sup></h3>

<h5><u>CREATING THE FACT TABLE WITH THE REQUIRED FIELDS (CONTAINING THE NATURAL KEYS)</u></h4>

In [9]:
fact_order_line = pd.merge(orders, items, on = 'order_id')

In [10]:
order_line_quantity = fact_order_line.groupby(['order_id', 'product_id']).size().reset_index(name='Order Line Quantity')

In [11]:
fact_order_line = pd.merge(fact_order_line, order_line_quantity, on = ['order_id', 'product_id'])

In [12]:
fact_order_line['Order Line Number'] = fact_order_line.groupby('order_id')['product_id'].rank(method='dense').astype(int)

In [13]:
fact_order_line[['Purchase Date',
                 'Approval Date',
                 'Delivery Date']] = fact_order_line[['order_purchase_timestamp', 
                                                      'order_approved_at', 
                                                      'order_delivered_customer_date']].apply(pd.to_datetime)

In [14]:
fact_order_line['Delivery Time'] = (fact_order_line['Delivery Date'] - fact_order_line['Approval Date']).dt.days

In [15]:
fact_order_line[['Purchase Date',
                 'Approval Date', 
                 'Delivery Date']] = fact_order_line[['Purchase Date', 
                                                      'Approval Date', 
                                                      'Delivery Date']].apply(lambda x: x.dt.date)

In [16]:
fact_order_line = fact_order_line.merge(customers[['customer_id', 'customer_zip_code_prefix']], on='customer_id')

In [17]:
fact_order_line = fact_order_line.merge(sellers, on='seller_id')

In [18]:
fact_order_line.drop(['order_purchase_timestamp', 'order_approved_at','order_delivered_customer_date',
                      'order_status', 'order_delivered_carrier_date', 'order_estimated_delivery_date',
                      'order_item_id', 'shipping_limit_date'], axis=1, inplace=True)

In [19]:
fact_order_line.duplicated().any()

True

In [20]:
fact_order_line = fact_order_line.drop_duplicates().reset_index(drop=True)

<h5><u>CREATING THE DEGENERATE DIMENSION</u></h5>

<h5>Degenerate Dimension<sup> dimOrderNumber</sup></h5>

In [21]:
dim_order_number = fact_order_line.loc[:, 'order_id'].to_frame()
dim_order_number.columns = ['ORDER_ID']

In [22]:
dim_order_number.duplicated().any()

True

In [23]:
dim_order_number = dim_order_number.drop_duplicates().reset_index(drop=True)

In [24]:
add_surrogate_key_dimension_table(dim_order_number, 'ORDER_NUMBER')

In [25]:
order_number_key_mapping = map_surrogate_to_natural_key(dim_order_number, 'ORDER_ID', 'ORDER_NUMBER')

In [26]:
dim_order_number

,ORDER_NUMBER,ORDER_ID
0,1,e481f51cbdc54678b7cc49136f2d6af7
1,2,53cdb2fc8bc7dce0b6741e2150273451
2,3,47770eb9100c2d0c44946d9cf07ec65d
3,4,949d5b44dbf5de918fe9c16f97b45f8a
4,5,ad21c59c0840e6cb83a9ceb5573f8159
...,...,...
96456,96457,9c5dedf39a927c1b2549525ed64a053c
96457,96458,63943bddc261676b46f01ca7ac2f7bd8
96458,96459,83c1379a015df1e13d02aae0204711ab
96459,96460,11c177c8e97725db2631073c19f07b62


In [27]:
print(f'The dimOrderNumber has {dim_order_number.isnull().sum().sum()} missing values.')

The dimOrderNumber has 0 missing values.


In [28]:
save_lookup_to_json(order_number_key_mapping, os.path.join(lookup_tables_folder, lookup_table_order_number))

<h5><u>SUBSTITUTING THE NATURAL KEYS BY THEIR SURROGATE COUNTERPARTS</u></h5>

In [29]:
fact_order_line.dtypes

order_id                     object
customer_id                  object
product_id                   object
seller_id                    object
price                       float64
freight_value               float64
Order Line Quantity           int64
Order Line Number             int32
Purchase Date                object
Approval Date                object
Delivery Date                object
Delivery Time                 int64
customer_zip_code_prefix      int64
seller_zip_code_prefix        int64
dtype: object

In [30]:
fact_order_line['customer_zip_code_prefix'] = fact_order_line['customer_zip_code_prefix'].astype(str)
fact_order_line['seller_zip_code_prefix'] = fact_order_line['seller_zip_code_prefix'].astype(str)
fact_order_line['Purchase Date'] = fact_order_line['Purchase Date'].astype(str)
fact_order_line['Approval Date'] = fact_order_line['Approval Date'].astype(str)
fact_order_line['Delivery Date'] = fact_order_line['Delivery Date'].astype(str)

In [31]:
fact_order_line = add_surrogate_key_fact_table(fact_order_line, os.path.join(lookup_tables_folder, lookup_table_order_number), 
                                               'order_id', 'ORDER_NUMBER')

In [32]:
fact_order_line = add_surrogate_key_fact_table(fact_order_line, os.path.join(lookup_tables_folder, lookup_table_customer), 
                                               'customer_id', 'CUSTOMER_KEY')

In [33]:
fact_order_line = add_surrogate_key_fact_table(fact_order_line, os.path.join(lookup_tables_folder, lookup_table_customer_geolocation), 
                                               'customer_zip_code_prefix', 'CUSTOMER_GEOLOCATION_KEY')

In [34]:
fact_order_line = add_surrogate_key_fact_table(fact_order_line, os.path.join(lookup_tables_folder, lookup_table_seller), 
                                               'seller_id', 'SELLER_KEY')

In [35]:
fact_order_line = add_surrogate_key_fact_table(fact_order_line, os.path.join(lookup_tables_folder, lookup_table_seller_geolocation), 
                                               'seller_zip_code_prefix', 'SELLER_GEOLOCATION_KEY')

In [36]:
fact_order_line = add_surrogate_key_fact_table(fact_order_line, os.path.join(lookup_tables_folder, lookup_table_product), 
                                               'product_id', 'PRODUCT_KEY')

In [37]:
fact_order_line = add_surrogate_key_fact_table(fact_order_line, os.path.join(lookup_tables_folder, lookup_table_date), 
                                               'Purchase Date', 'PURCHASE_DATE_KEY')

In [38]:
fact_order_line = add_surrogate_key_fact_table(fact_order_line, os.path.join(lookup_tables_folder, lookup_table_date), 
                                               'Approval Date', 'APPROVAL_DATE_KEY')

In [39]:
fact_order_line = add_surrogate_key_fact_table(fact_order_line, os.path.join(lookup_tables_folder, lookup_table_date), 
                                               'Delivery Date', 'DELIVERY_DATE_KEY')

In [40]:
fact_order_line.rename(columns={'price': 'UNIT_PRICE',
                                'freight_value': 'FREIGHT_COST',
                                'Order Line Quantity': 'ORDER_LINE_QUANTITY',
                                'Order Line Number': 'ORDER_LINE_NUMBER',
                                'Delivery Time': 'DELIVERY_TIME'}, inplace=True)

In [41]:
fact_order_line = fact_order_line[['ORDER_NUMBER', 'ORDER_LINE_NUMBER', 'CUSTOMER_KEY', 'CUSTOMER_GEOLOCATION_KEY', 'SELLER_KEY', 
                                   'SELLER_GEOLOCATION_KEY', 'PRODUCT_KEY', 'PURCHASE_DATE_KEY', 'APPROVAL_DATE_KEY', 
                                   'DELIVERY_DATE_KEY', 'ORDER_LINE_QUANTITY', 'UNIT_PRICE', 'FREIGHT_COST', 'DELIVERY_TIME']]

In [42]:
fact_order_line = fact_order_line.sort_values('ORDER_NUMBER').reset_index(drop=True)

In [43]:
fact_order_line

,ORDER_NUMBER,ORDER_LINE_NUMBER,CUSTOMER_KEY,CUSTOMER_GEOLOCATION_KEY,SELLER_KEY,SELLER_GEOLOCATION_KEY,PRODUCT_KEY,PURCHASE_DATE_KEY,APPROVAL_DATE_KEY,DELIVERY_DATE_KEY,ORDER_LINE_QUANTITY,UNIT_PRICE,FREIGHT_COST,DELIVERY_TIME
0,1,1,70297,1048,560,4130,2350,641,641,649,1,29.99,8.72,8
1,2,1,77028,10823,550,8235,20551,936,938,950,1,118.70,22.76,12
2,3,1,555,15301,2618,5454,12229,951,951,960,1,159.90,19.22,9
3,4,1,61082,12334,2990,8269,29926,688,688,702,1,45.00,27.20,13
4,5,1,67264,4098,1497,4022,11901,775,775,778,1,19.90,8.72,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100175,96457,1,60578,4593,2577,4784,18469,434,434,442,1,72.00,13.08,8
100176,96458,1,78425,4533,36,5946,1779,768,768,790,1,174.90,20.10,22
100177,96459,1,19902,10666,2114,3827,14954,605,605,630,1,205.99,65.02,24
100178,96460,1,16924,7699,2747,9272,8291,739,739,756,2,179.99,40.59,17


In [44]:
print(f'The factOrderLine dataframe has {fact_order_line.isnull().sum().sum()} missing values.')

The factOrderLine dataframe has 0 missing values.


<h3>Fact Table <sup>factOrderHeaderAccumulating</sup></h3>

<h5><u>CREATING THE FACT TABLE WITH THE REQUIRED FIELDS (CONTAINING THE NATURAL KEYS)</u></h5>

In [45]:
fact_order_header = pd.merge(orders, items, on = 'order_id')

In [46]:
fact_order_header.drop(['order_delivered_carrier_date', 'order_estimated_delivery_date', 'product_id', 
                        'seller_id', 'shipping_limit_date', 'price', 'freight_value'], axis=1, inplace=True)

In [47]:
order_item_quantity_df = fact_order_header.drop(['customer_id', 'order_status', 'order_purchase_timestamp', 
                                                 'order_approved_at', 'order_delivered_customer_date'], axis=1)

In [48]:
order_item_quantity_df = order_item_quantity_df.groupby('order_id', as_index=False).count()

In [49]:
order_item_quantity_df = order_item_quantity_df.rename(columns={'order_item_id': 'order_item_quantity'})

In [50]:
fact_order_header = fact_order_header.drop_duplicates(subset='order_id')

In [51]:
fact_order_header = pd.merge(fact_order_header, order_item_quantity_df, on='order_id')

In [52]:
payment_value_by_order_id = payments.groupby('order_id')['payment_value'].sum().reset_index()

In [53]:
new_payments = payments.copy()

In [54]:
new_payments = new_payments.loc[new_payments.groupby('order_id')['payment_value'].idxmax()]

In [55]:
new_payments.drop(['payment_sequential', 'payment_installments'], axis=1, inplace=True)

In [56]:
fact_order_header = pd.merge(fact_order_header, new_payments, on='order_id')

In [57]:
fact_order_header['order_indicator'] = fact_order_header['payment_type'] + ' + ' + fact_order_header['order_status']

In [58]:
fact_order_header[['Purchase Date',
                   'Approval Date', 
                   'Delivery Date']] = fact_order_header[['order_purchase_timestamp', 
                                                          'order_approved_at', 
                                                          'order_delivered_customer_date']].apply(pd.to_datetime)

In [59]:
fact_order_header.drop(['payment_type', 'order_status', 'order_item_id', 'order_purchase_timestamp', 
                        'order_approved_at', 'order_delivered_customer_date'], axis=1, inplace=True)

In [60]:
fact_order_header['Delivery Time'] = (fact_order_header['Delivery Date'] - fact_order_header['Approval Date']).dt.days

In [61]:
fact_order_header[['Purchase Date',
                   'Approval Date', 
                   'Delivery Date']] = fact_order_header[['Purchase Date', 
                                                          'Approval Date', 
                                                          'Delivery Date']].apply(lambda x: x.dt.date)

In [62]:
mean_review_score_by_order_id = reviews.groupby('order_id')['review_score'].mean().reset_index()

In [63]:
mean_review_score_by_order_id['review_score'] = mean_review_score_by_order_id['review_score'].astype(int)

In [64]:
fact_order_header = pd.merge(fact_order_header, mean_review_score_by_order_id, on='order_id', how='left')

In [65]:
print(f'The factOrderHeader dataframe has {fact_order_header.isnull().sum().sum()} missing values.')

The factOrderHeader dataframe has 646 missing values.


In [66]:
fact_order_header['review_score'].isna().sum().sum()

646

In [67]:
fact_order_header['review_score'] = fact_order_header['review_score'].fillna(0)

In [68]:
print(f'The factOrderHeader dataframe now has {fact_order_header.isnull().sum().sum()} missing values.')

The factOrderHeader dataframe now has 0 missing values.


In [69]:
fact_order_header = fact_order_header.merge(customers[['customer_id', 'customer_zip_code_prefix']], on='customer_id')

<h5><u>SUBSTITUTING THE NATURAL KEYS BY THEIR SURROGATE COUNTERPARTS</u></h5>

In [70]:
fact_order_header.dtypes

order_id                     object
customer_id                  object
order_item_quantity           int64
payment_value               float64
order_indicator              object
Purchase Date                object
Approval Date                object
Delivery Date                object
Delivery Time                 int64
review_score                float64
customer_zip_code_prefix      int64
dtype: object

In [71]:
fact_order_header['customer_zip_code_prefix'] = fact_order_header['customer_zip_code_prefix'].astype(str)
fact_order_header['Purchase Date'] = fact_order_header['Purchase Date'].astype(str)
fact_order_header['Approval Date'] = fact_order_header['Approval Date'].astype(str)
fact_order_header['Delivery Date'] = fact_order_header['Delivery Date'].astype(str)

In [72]:
fact_order_header = add_surrogate_key_fact_table(fact_order_header, os.path.join(lookup_tables_folder, lookup_table_order_number), 
                                                 'order_id', 'ORDER_NUMBER')

In [73]:
fact_order_header = add_surrogate_key_fact_table(fact_order_header, os.path.join(lookup_tables_folder, lookup_table_customer), 
                                                 'customer_id', 'CUSTOMER_KEY')

In [74]:
fact_order_header = add_surrogate_key_fact_table(fact_order_header, os.path.join(lookup_tables_folder, lookup_table_customer_geolocation), 
                                                 'customer_zip_code_prefix', 'CUSTOMER_GEOLOCATION_KEY')

In [75]:
fact_order_header = add_surrogate_key_fact_table(fact_order_header, os.path.join(lookup_tables_folder, lookup_table_date), 
                                                 'Purchase Date', 'PURCHASE_DATE_KEY')

In [76]:
fact_order_header = add_surrogate_key_fact_table(fact_order_header, os.path.join(lookup_tables_folder, lookup_table_date), 
                                                 'Approval Date', 'APPROVAL_DATE_KEY')

In [77]:
fact_order_header = add_surrogate_key_fact_table(fact_order_header, os.path.join(lookup_tables_folder, lookup_table_date), 
                                                 'Delivery Date', 'DELIVERY_DATE_KEY')

In [78]:
fact_order_header = add_surrogate_key_fact_table(fact_order_header, os.path.join(lookup_tables_folder, lookup_table_order_indicator), 
                                                 'order_indicator', 'ORDER_INDICATOR_KEY')

In [79]:
fact_order_header.rename(columns={'order_item_quantity': 'ORDER_ITEM_QUANTITY',
                                  'payment_value': 'ORDER_AMOUNT',
                                  'Delivery Time': 'DELIVERY_TIME',
                                  'review_score': 'REVIEW_SCORE'}, inplace=True)

In [80]:
fact_order_header = fact_order_header[['ORDER_NUMBER', 'CUSTOMER_KEY', 'CUSTOMER_GEOLOCATION_KEY', 'PURCHASE_DATE_KEY',
                                       'APPROVAL_DATE_KEY', 'DELIVERY_DATE_KEY', 'ORDER_INDICATOR_KEY', 'ORDER_AMOUNT', 
                                       'ORDER_ITEM_QUANTITY', 'REVIEW_SCORE', 'DELIVERY_TIME']]

In [81]:
fact_order_header = fact_order_header.sort_values('ORDER_NUMBER').reset_index(drop=True)

In [82]:
fact_order_header

,ORDER_NUMBER,CUSTOMER_KEY,CUSTOMER_GEOLOCATION_KEY,PURCHASE_DATE_KEY,APPROVAL_DATE_KEY,DELIVERY_DATE_KEY,ORDER_INDICATOR_KEY,ORDER_AMOUNT,ORDER_ITEM_QUANTITY,REVIEW_SCORE,DELIVERY_TIME
0,1,70297,1048,641,641,649,5,18.59,1,4.0,8
1,2,77028,10823,936,938,950,3,141.46,1,4.0,12
2,3,555,15301,951,951,960,1,179.12,1,5.0,9
3,4,61082,12334,688,688,702,1,72.20,1,5.0,13
4,5,67264,4098,775,775,778,1,28.62,1,5.0,2
...,...,...,...,...,...,...,...,...,...,...,...
96455,96457,60578,4593,434,434,442,1,85.08,1,5.0,8
96456,96458,78425,4533,768,768,790,1,195.00,1,4.0,22
96457,96459,19902,10666,605,605,630,1,271.01,1,5.0,24
96458,96460,16924,7699,739,739,756,1,441.16,2,2.0,17


In [83]:
print(f'The factOrderHeader dataframe has {fact_order_header.isnull().sum().sum()} missing values.')

The factOrderHeader dataframe has 0 missing values.


<h3>4. EXPORTING THE CSV FILES OF THE FACTS</h3>

In [84]:
os.makedirs(fact_tables_folder, exist_ok=True)

In [85]:
fact_order_line.to_csv(os.path.join(fact_tables_folder, fact_order_line_file), index=False)
fact_order_header.to_csv(os.path.join(fact_tables_folder, fact_order_header_file), index=False)